# Genki Signals Mockup

Some rough ideas for how the api will look and feel.

This library consists of four entities:
* `SignalSource`: A class which defines how raw data is read. Referred to as a "signal".
    * This data can have any dimensionality but the last axis must be the time axis.
    * e.g. the MouseDataSource is a 2d signal with dimensions (2, T). It captures the x and y positions of your mouse on the computer screen.
* `SignalFunction`: A class which defines operations on real-time data (i.e. signals) which produces new signals, sometimes referred to as "derived signals". 
    * e.g. the Add operation can add together two signals assuming that their dimensionality matches.
* `SignalManager`: A class which encapsulates `SignalSources` and `SignalFunctions` units, allowing you to skip most of the boilerplate and record data.
* `SignalInspector`: An abstract class which receives data from the `SignalManager` via callback. It has two modes, <i>live</i> and <i>static</i>.
    * The <i>static</i> mode, i.e. viewing data you have already recorded, has the following:
        * The `Analyzer` class is for visualzing information on entire time series.
        * The `Labeller` class has ux so that the user can label their data, written as an annotation file.
    * The <i>live</i> mode, e.g. viewing model inference in real time, has the following:
        * The `Visualizer` can display real-time data with many different types of plots, e.g. spectrogram, trace, line and barplots.


In [3]:
import os 
os.chdir('/Users/bjarnihaukurbjarnason/Documents/genki-signals')

from genki_signals.signal_sources import MouseSource, Sampler 

## SignalSource

We have an extra distinction for types of `SignalSources`, one being sources that solely define <i>how</i> data is accessed the other being sources that define <i>when</i> data is accessed. (Some sources can be both)

An excellent example for the former type is the `MouseSource` which simply defines how the mouse positions are read.
```python
class MouseSource(SignalSource):
    def __init__(self):
        import pynput

        self.mouse = pynput.mouse.Controller()

    def __call__(self, t):
        return np.array(self.mouse.position)
```
This however does not create real-time data on its own and so we need to wrap it in the latter type of source, e.g. the `Sampler` class.
```python
class Sampler(SamplerBase):
    def __init__(self, sources, sample_rate, sleep_time=1e-6, timestamp_key="timestamp"):
        ...

source = Sampler([MouseDataSource()], 100)
```
In this example the `Sampler` "samples" from the `MouseSource` 100 times a second. Let's see this done in practice.

In [ ]:
import time

# Initialize the sampler with a single data source and a sample rate of 100 Hz
source = Sampler(sources=[MouseSource()], sample_rate=100)

source.start() # Starts collecting data

time.sleep(1)

data = source.read()
print(len(data)) # Since we waited for 1 second, we should around 100 samples
print(data.keys()) # The data is stored in a dictionary with the keys being the names of the data sources, should print ['timestamp, 'mouse_position']

source.stop() # Stops collecting data

## SignalFunctions

Next in line are the so called "derived signals" which are created deterministically with the `SignalProcessing` classes. Let's see how they work

In [ ]:
from genki_signals import signal_functions as sf

input_signal = 'mouse_position'
name = 'mouse_position_doubled'
mult = sf.Scale(
    input_signal=input_signal, # This is used later on with the SignalManager class
    scale_factor=2,
    name=name, # This is used later on with the SignalManager class
) 

# Let's collect data like before
source.start()

time.sleep(1)

data = source.read()
doubled_position = mult(data[input_signal])
print(all(data[input_signal] * 2 == doubled_position)) # Should print True

source.stop()

This however is not nice to work with and so we come to the third entity mentioned, the `SignalManager` class.

## SignalManager

The `SignalManager` class handles all the boilerplate showed previously and more. To put it simply, the manager starts a seperate thread where data is read and derived signals computed at the provided sampling rate.

In [ ]:
from genki_signals import SignalManager
from genki_signals.signal_sources import MouseSource, Sampler 
from genki_signals import signal_functions as sf

source = Sampler(sources=[MouseSource()], sample_rate=100)
derived = [
    sf.Scale('mouse_position', scale_factor=2, name='mouse_position_doubled'),
    sf.Add('mouse_position', 'mouse_position_doubled', name='mouse_position_tripled'),
]

manager = SignalManager(source, derived, sample_rate=10) # The sample rate of the SignalManager is independent of the sample rate of the data source

manager.start() # Starts collecting data and computing derived signals, each time a new sample is collected it should contain 100/10=10 samples

To summarize, the system contains the `SignalSources` and `SignalFunctions`, and when we read new data it sends data corresponding to 'mouse_position' to `Scale` which outputs 'mouse_position_doubled' which is then an input to `Add` along with the original 'mouse_position' producing 'mouse_position_tripled'

In [ ]:
manager.stop()

In [ ]:
from genki_signals import SignalManager
from genki_signals.signal_sources import MouseSource, Sampler 
from genki_signals import signal_functions as sf
from genki_signals.gui import BqWidgetGUI

source = Sampler(sources=[MouseSource()], sample_rate=100)
derived = [
    sf.Derivative('mouse_position', name='mouse_velocity'),
]

gui_frontend = BqWidgetGUI()

manager = SignalManager(
    signal_source=source,
    derived_signals=derived,
    sample_rate=10, # The sample rate of the SignalManager is independent of the sample rate of the data source
    frontends = [gui_frontend] # can be appended later with .add_frontend()
)

manager.start() # Starts collecting data and computing derived signals, each time a new sample is collected it should contain 100/10=10 samples

We create plots on our real-time data on the frontend object

In [ ]:
# VIP
vel_plot = gui_frontend.plot(
    x_key='timestamp',
    y_key='mouse_velocity',
    x_label='Time',
    y_label='Velocity',
    title='Mouse velocity',
    n_visible_points=100
)

pos_trace = gui_frontend.trace('mouse_position', n_visible_points=100)

In [ ]:
vel_plot

In [ ]:
pos_trace